# Tutorial 1: Simple Volume Rendering with yt

This notebook provides an example of how to prepare Rayleigh Spherical_3D data output for volume rendering with yt. Then we show a few simple examples of how to create volume renderings, modify the transfer function, and manipulate the camera angle. 

## About yt and installing yt
*This section is under development*

Developed in Python, yt is an analysis and visualization toolkit for volumetric data. To install yt on your machine and activate it, visit this website: [https://yt-project.org/doc/installing.html](https://yt-project.org/doc/installing.html) 

## Preparing your Rayleigh Spherical_3D output for volume rendering with yt
*This section is under development*

Rayleigh's Spherical_3D quantities are output in a spherical grid. Many visualization software packages do not fully support spherical grids. One solution is to interpolate the quantities onto a uniform Cartesian grid with dimensions XxYxZ. Quantity values outside the spherical volume are given values of zero and can easily be made transparent using an appropriately chosen tranfer function when rendering the visualization.

To prepare your Spherical_3D output: Interpolating Spherical_3D quantities onto a uniform Cartesian grid

1. Copy the directory **/Rayleigh/post_processing/kyle_interp** onto your local system
2. Compile the executable. We use the gcc compiler: **make -f Makefile.gfortran** 
3. Create a directory called **vis_test** (or something equally suitable) in which to run the interpolator. Copy the executable **interpolator.app** and the **input** file from **/Rayleigh/post_processing/kyle_interp** there.
4. Create a directory in **vis_test** called **Spherical_3D**. In directory **Spherical_3D** copy the 3D spherical output files you wish to interpolate, including the **_grid** file.
5. In the input file, modify the uniform radial grid resolution **nr** appropriate for your files, choose the resolution **ncube** of the resulting Cartesian cube, and modify the iteration range. When interpolating a single file, **initial_iteration** and **final_iteration** should be the same.
6. Run the interpolation code: **./interpolator.app XXXX** (Where **XXXX** is the four digit quantity code of the quantity you wish to interpolate. For example, to interpolate a radial velocity spherical 3D output of quantity code **0001**, type: **./interpolator.app 0001**) 
7. If the code executed correctly, you should have a new file in your **/vis_test/Spherical_3D** directory called **_cube**. These files are what you will import into this notebook. 
8. Copy your new **_cube** files into the directory where you will run this Jupyter Notebook.


## Reading in your prepared data and creating simple yt visualizations
*This section is under development. There are known issues with YT that need to be resolved.*

After (1) your yt installation has been activated and (2) you have interpolated your Rayleigh Spherical_3D output onto a uniform cartesian grid, you are ready to read the data into Python and begin visualizations with yt. 

In [ ]:
import numpy
from rayleigh_diagnostics import Spherical_3D

#This reads in one of the cubed data files
#All cube data has been interpolated onto a uniform cartesian grid with dimensions 128x128x128
rfile = 'vr_cube' #The name of your cubed data file - here we are looking at radial velocity
nx=128
ndata=nx**3
arr = numpy.fromfile(rfile,dtype='float64')
cube = numpy.reshape(arr,(nx,nx,nx))

Load the data into YT for volume rendering and create a YT scene.

In [ ]:
import yt
import numpy as np

#Render an image of the cartesian grid data
#Note that the negative values will not be plotted here. This is an issue that needs to be resolved.

data = dict(velocity = (cube, "cm/s"))
bbox = numpy.array([[-1.5, 1.5], [-1.5, 1.5], [-1.5, 1.5]]) #Define a bounding box 
ds = yt.load_uniform_grid(data, cube.shape, bbox=bbox, nprocs=nx) #Load a cube into YT
sc = yt.create_scene(ds, field=('velocity')) #Set up a basic Scene in YT
maxup = np.amax(cube)
minup = np.amin(cube)
print (maxup, minup) #Print the max and min values of the data

Show the volume rendering and plot the default transfer function.

In [ ]:
sc.show() #Shows the image. Note that negative values will not be rendered. This is an issue that needs to be resolved.
source = sc[0]
source.tfh.plot(profile_field='velocity') #Plots the transfer function and the field profile

Let's try to create a new tranfer function that we define ourselves. The method outlined below is closer to that of an isosurface render.

In [ ]:
#This is not an optimal transfer function, just an example of what you can do
#More detailed transfer functions can be defined

source.set_field('velocity') #Set the field to be rendered
source.set_log(False) #Enforce a linear space - appears to work for the transfer function, but not for volume render
bounds = (0, 12) #Set the bounds of the colormap in data space

tf = yt.ColorTransferFunction(bounds) #Define a new transfer function

#This is how to add new opacity points to the colormap 'plasma' with a gaussian shape
tf.sample_colormap(1., w=1., colormap='plasma') #Gaussian centered at 1 and width 1
tf.sample_colormap(5., w=1., colormap='plasma') #Gaussian centered at 5 and width 1
tf.sample_colormap(10., w=1., colormap='plasma') #Gaussian centered at 10 and width 1

source.tfh.tf = tf
source.tfh.bounds = bounds

source.tfh.plot(profile_field='velocity') #Plots the tranfer function

Show the image with our newly defined transfer function.

In [ ]:
#Grab the first render source, set it to use the new transfer function
render_source = sc.get_source()
render_source.transfer_function = tf

sc.render()
sc.show(sigma_clip=2.0) #Attempt a better contrast with sigma clipping

Below is an example of how you can (1) save the images, and (2) rotate the camera around the image 180 degrees in 5 iterations to see it from different viewpoints in longitude.

In [ ]:
# save an image at the starting position 
frame = 0
sc.save('camera_movement_%04i.png' % frame)
frame += 1

# rotate the image in azimuth 180 degrees over 5 iterations, showing and saving each frame
for _ in sc.camera.iter_rotate(np.pi, 5, rot_vector=np.array([1.0, 0.0, 0.0])):
    sc.render()
    sc.show(sigma_clip=2.0)
    #sc.save('camera_movement_%04i.png' % frame) #Uncomment this line if you want to save your series of images.
    frame += 1

Here is another way to create custom transfer functions. It is possible to define your own functions to map out a custom opacity level. This one creates a transfer function that increases linearly in opacity over the defined bounds, using all the colors available in the entire color map. Find out more about tranfer function capabilities [here](https://yt-project.org/doc/reference/api/yt.visualization.volume_rendering.transfer_functions.html?highlight=map_to_colormap#module-yt.visualization.volume_rendering.transfer_functions).

In [ ]:
#Another example of defining your own transfer function with a user-defined function, here called linramp
#Note again that this is not an optimal transfer function, just an example of what you can do

source = sc[0]

source.set_field('velocity')
source.set_log(False)

bounds = (0., 12.) #set the bounds of your transfer function in data values

tf = yt.ColorTransferFunction(bounds)

def linramp(vals, minval, maxval): #User-defined function to control the alpha levels, alpha is similar to opacity 
    return (vals - vals.min())/(vals.max() - vals.min())

tf.map_to_colormap(0., 12., colormap='Rainbow', #scale_func(function(value, minval, maxval))
                   scale_func=linramp)

source.tfh.tf = tf
source.tfh.bounds = bounds

source.tfh.plot(profile_field='velocity') #Plot the new transfer function

Show the image with our newly defined transfer function. This transfer function will highlight the strongest positive radial velocity structures. 

In [ ]:
# Grab the first render source and set it to use the new transfer function
render_source = sc.get_source()
render_source.transfer_function = tf

sc.render()
sc.show(sigma_clip=4.0) #Render the image and show it on screen